# Pytorch Tutorial

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
from tqdm import tqdm

import torch
import torch.nn as nn
#from torch.autograd import Variable

#import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

In [3]:
def set_seed(RANDOM_SEED=42):
    torch.manual_seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    import random
    random.seed(RANDOM_SEED)
    
set_seed()

# 0. Data upload

일반적인 tutorial을 보면 torchvision과 같은 패키지에서 tutorial data를 쉽게 불러옵니다.

하지만 real data를 다루게 될 경우 다른 방식으로 전처리하셔야 합니다.

그래서 나중에 참고하실 tutorial 방식과 약간 다를 수 있습니다만 더 먼 미래에 pytorch로 여러분의 코드를 만드실 때에는 밑에서 다루는 것처럼 custom dataset을 만드시게 됩니다.

그래서 일단 github에서 csv파일 형태로 data를 받아오겠습니다.

In [1]:
!git clone https://github.com/yeonseok-jeong-cm/yeonseok_fashion_mnist

Cloning into 'yeonseok_fashion_mnist'...


In [4]:
with gzip.open('./yeonseok_fashion_mnist/fashion-mnist_train.gz') as f:

    train_csv = pd.read_csv(f)
    
with gzip.open('./yeonseok_fashion_mnist/fashion-mnist_test.gz') as f:

    test_csv = pd.read_csv(f)

In [5]:
train_csv

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


-----

# 1. DataLoader

## 0) DataLoader에 대한 소개

<font color="red">**DataLoader**</font>는 pytorch에서 핵심적인 개념입니다. 

제가 공개한 파일(iterator_generator.ipynb)을 보면 

generator 혹은 iterator는 필요한 만큼 data를 추출하고 버릴 수 있습니다. (pop)

그래서 효율적으로 data를 관리할 수 있다.

Deep Learning에서 이를 <font color="blue">**mini-batch**</font> 라고 합니다. (batch_size으로도 불립니다.)

## 1) Custom Dataset

#### 1> 내부 data는 Tensor 형태로 만든다. 

- [1] 먼저 numpy 형태의 dataset을 만든 뒤 (image는 3d-array 꼴로 만든다.)
(W, H, C)
- [2] `torchvision.transform.ToTensor`를 이용해서 Tensor로 변환한다.

#### 2> 아래의 메서드를 현재 dataset에 맞게 만든다.

- [1] `__init__` : 생성자
- [2] `__getitem__(self, index)` : indexing 기능을 구현
- [3] `__len__(self)` : data의 개수를 return (by `len`)

In [6]:
class FashionDataset(torch.utils.data.Dataset): # Dataset 상속 받아야함
    
    def __init__(self, data, transform = None):
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        # 1-0> csv data를 row 하나씩 list에 저장
        for i in self.fashion_MNIST:
            label.append(i[0]) # label
            image.append(i[1:]) # image data -> 1d => 2d(append 효과로)
        # 1-1> numpy dataset 생성 (3d-array가 모여 4d-array 형태)
        self.labels = np.asarray(label)
        self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32') # Gray Scaling 효과(채널은 1개)
        # reshape(-1,28,28,1) => (data,W,H,C)
    def __getitem__(self, index):
        # 2-2> indexing 기능을 구현
        label = self.labels[index]
        image = self.images[index]
        
        # 1-2> 추후 ToTensor로 numpy dataset을 Tensor dataset으로 변환
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        # 2-3> data 개수를 return
        return len(self.images)

In [7]:
train_set = FashionDataset(train_csv, transform=transforms.Compose([transforms.ToTensor()]))
test_set = FashionDataset(test_csv, transform=transforms.Compose([transforms.ToTensor()]))

In [8]:
train_set.images.shape

(60000, 28, 28, 1)

## 2) Sampler

https://pytorch.org/docs/stable/data.html#torch.utils.data.Sampler

<font color="red">**Sampler**</font> : dataset에서 mini-batch만큼 추출할 때 (비복원 추출) 어떠한 규칙으로 추출할지

mini-batch(표본) 내부 구성이 다양할수록 전체 dataset(모집단)를 잘 대표하기 때문에 주로 RandomSampler를 사용합니다.

In [12]:
from torch.utils.data import RandomSampler

In [13]:
train_random_sampler = RandomSampler(train_set)
test_random_sampler = RandomSampler(test_set) 

## 3) DataLoader

https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

#### 1> DataLoader에서 설정할 것

- dataset
- batch_size
- sampler
 
 
#### 2> DataLoader에 대한 간단한 설명

<font color="red">**DataLoader**</font>는 
- [1] 저희가 만든 <font color="blue">**dataset**</font>을
- [2] <font color="blue">**batch_size**</font>만큼 (필요한 만큼)
- [3] <font color="blue">**sampler**</font>라는 규칙으로 data를 추출해주는 class

In [14]:
BATCH_SIZE = 32

In [15]:
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, sampler=train_random_sampler)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, sampler=test_random_sampler)

#### 3> DataLoader 내부를 `iter`와 `next`를 이용해서 보겠습니다.

In [16]:
train_iterator = iter(train_loader)

In [17]:
dir(train_iterator) # 안에 함수 뭐가 있는지 알 수 있는 명령어 dir

['_IterableDataset_len_called',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_base_seed',
 '_collate_fn',
 '_dataset',
 '_dataset_fetcher',
 '_dataset_kind',
 '_drop_last',
 '_index_sampler',
 '_next_data',
 '_next_index',
 '_num_workers',
 '_num_yielded',
 '_persistent_workers',
 '_pin_memory',
 '_prefetch_factor',
 '_reset',
 '_sampler_iter',
 '_timeout',
 'next']

In [18]:
a1 = next(train_iterator)

In [19]:
len(a1)

2

In [21]:
a1[0].shape # batch size ,channel,width,height

torch.Size([32, 1, 28, 28])

In [23]:
a1[0] # image
a1[1] # label

tensor([4, 7, 5, 0, 7, 7, 3, 2, 2, 2, 4, 7, 7, 0, 8, 3, 7, 2, 7, 2, 1, 7, 8, 2,
        5, 7, 4, 4, 9, 4, 5, 0])

## cf> label mapping

In [24]:
def output_label(label):
    output_mapping = {
                 0: "T-shirt/Top",
                 1: "Trouser",
                 2: "Pullover",
                 3: "Dress",
                 4: "Coat", 
                 5: "Sandal", 
                 6: "Shirt",
                 7: "Sneaker",
                 8: "Bag",
                 9: "Ankle Boot"
                 }
    input = (label.item() if type(label) == torch.Tensor else label)
    return output_mapping[input]

-----

# 2. Build the Model

모델 내부에 존재하는 layer들은 아직 배우지 않은 것들이 있으며 

framework 소개와는 관련성이 떨어지므로 설명을 생략하겠습니다.

## 1) `__init__`
Network 내부에서 사용할 구조들을 모두 만듭니다.

### cf> Sequential

Sequential의 경우 원래 keras에 존재하는 편리하게 모델을 만드는 class입니다. (keras의 강력한 장점 중 하나입니다.)

강의 초반에 언급했듯이 각 Framework들은 서로의 장점을 닮아가는 특성상 

pytorch도 이 Sequential을 버전이 업데이트 되면서 Sequential을 가져왔습니다.

## 2) `forward`

forward propagation을 구현합니다. 

### cf> Train vs validation(Test)

Train : forward propagation, compute loss, backpropagation, gradient descent

Test : forward propagation

위와 같은 이유로 Neural Network class에서는 forward propagation까지만 구현합니다.

In [25]:
class FashionCNN(nn.Module): # 모듈 상속
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), # 배치 정규화
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600) # layer.Dense와 같음
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10) # class label 맞춤 (10개 클래스)
        
    def forward(self, x):
        out = self.layer1(x) # 
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # reshape 1d array
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

-----

# 3. Train & validation

## 0) Train 전에 준비할 것들

#### 1> CPU 혹은 GPU 확인

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

#### 2> build the model

In [27]:
model = FashionCNN()
model.to(device)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)

#### 3> Hyperparameter 

In [30]:
LEARNING_RATE = 0.001
EPOCHS = 3
BATCH_SIZE = 2**5

#### 4> Train에 필요한 연산 정의

- loss
- optimizer

In [31]:
error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
total_batch = len(train_loader)
print(model)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


## 1) Train & 2) Validation

In [32]:
## 1) Train

for epoch in range(EPOCHS):
    print('\n'+'*'*50)
    print(f'Epoch {epoch+1}')
    avg_cost = 0
    model.train() # 학습 선언
    for X, Y in tqdm(train_loader): # batch_size 단위로 꺼내온다.
        # 0> 연산하고자 하는 Tensor를 사용하는 device에 올려준다.
        X, Y = X.to(device), Y.to(device)       
        # 2> forward propagation
        hypothesis = model(X) # model.forward() == model(image)
        # 3> compute loss(cost) function
        cost = error(hypothesis, Y) # error(pred_y,true_y)
        ## backpropagation 단계 전에, Optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인) 갱신할 변수들에 대한 모든 변화도를 0으로 만듭니다.
        ## 이렇게 하는 이유는 기본적으로 .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고) 누적되기 때문입니다.
        optimizer.zero_grad() 
        # 4> backward propagation
        cost.backward()
        # 5> gradient descent
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: %d] train loss = %0.9f' % (epoch + 1, avg_cost))
# 2) validation
    correct = 0
    total = 0
    with torch.no_grad(): # Neural Network 연산 중에 gradient를 저장할 필요가 없으므로 
        model.eval() # validation mode
        for X_val, Y_val in tqdm(test_loader):
            # 0> 연산하고자 하는 Tensor를 사용하는 device에 올려준다.
            X_val, Y_val = X_val.to(device), Y_val.to(device)
            # 2> forward propagation
            hypothesis = model(X_val)
            # calculate accuracy
            _, predicted = torch.max(hypothesis, 1)
            total += Y_val.size(0)
            correct += (predicted == Y_val).sum().item()
            
    print('[Epoch: %d] val accuracy = %0.2f %%' % (epoch + 1, 100. * float(correct / total)))

  0%|                                                                                         | 0/1875 [00:00<?, ?it/s]


**************************************************
Epoch 1


 55%|███████████████████████████████████████████▍                                   | 172/313 [00:00<00:00, 846.99it/s]

[Epoch: 1] train loss = 0.440346479


  1%|█                                                                              | 25/1875 [00:00<00:07, 243.36it/s]

...

[Epoch: 2] val accuracy = 90.88 %

**************************************************
Epoch 3


 58%|█████████████████████████████████████████████▍                                 | 180/313 [00:00<00:00, 890.79it/s]

[Epoch: 3] train loss = 0.254831046


100%|███████████████████████████████████████████████████████████████████████████████| 313/313 [00:00<00:00, 884.04it/s]

[Epoch: 3] val accuracy = 90.11 %
